### Code Snippets for calling ImageJ ops

In [1]:
import imagej
import scyjava as sj
ij = imagej.init('sc.fiji:fiji:2.14.0')

In [105]:
op_name = "filter.median"
type_mapping_table_file = 'mapping_common_input_java_types_full.xlsx'

### Sub Methods

In [106]:
import textwrap

def import_part():
    return "import imagej\nimport scyjava as sj\nij = imagej.init('sc.fiji:fiji:2.14.0')\n"
print(import_part())

def get_input_image():
    return "#Get input image\nvp_input_image = n_1_image\nprocess_image = vp_input_image.get('value')\n"
print(get_input_image())

def get_op_name(op_name):
    return f"Op_name = '{op_name}'\n"
print(get_op_name(op_name))
                 
### Here we only take the first operation
### Modify this in multi-operation case
def is_empty_result_image_required(opInfo):
    for input in opInfo[0].inputs():
        if input.isOutput() and input.isRequired():
            return True
    return False
    
def prepare_empty_result_image():
    return "jimage = ij.py.to_java(process_image)\nresult_image = ij.op().run('create.img',jimage)\n"
print(prepare_empty_result_image())

def get_op_info(Op_name):
    ops = ij.op()
    all_infos = ops.infos()
    op_infos = [info for info in all_infos if info.getName() == Op_name]
    return op_infos

### Here we only take the first operation
### Modify this in multi-operation case
def get_op_required_parameters(opInfo):
    para_dict = {}
    for input in opInfo[0].inputs():
        if input.isRequired():
            current_input_type = str(input.getType())
            # Remove 'interface ' or 'class ' prefix from the type string
            if current_input_type.startswith('interface '):
                current_input_type = current_input_type[len('interface '):]
            elif current_input_type.startswith('class '):
                current_input_type = current_input_type[len('class '):]
            para_dict[input.getName()] = current_input_type
    return para_dict
op_parameters = get_op_required_parameters(get_op_info(op_name))

def get_short_classname(full_classname):
     return full_classname.split('.')[-1]
    
def import_java_class_by_jimport(op_parameters, mapping_dict):
    import_statements = ''
    for key, value in op_parameters.items():
        if value in mapping_dict and mapping_dict[value]['jimport'] not in {None, 'None'}:
            short_classname = get_short_classname(mapping_dict[value]['jimport'])
            import_statements += f'{short_classname} = sj.jimport("{mapping_dict[value]["jimport"]}")\n'
    return import_statements
import_java_class_by_jimport(op_parameters,mapping_dict)

def op_calling_and_running(opInfo, mapping_dict):                   
    code = 'result_image= ij.op().run(Op_name, ij.py.jargs('
    args = []
    for input in opInfo[0].inputs():
        if not input.isRequired():
            args.append('None')
            continue
            
        current_input_type = str(input.getType())
        if current_input_type.startswith('interface '):
            current_input_type = current_input_type[len('interface '):]
        elif current_input_type.startswith('class '):
            current_input_type = current_input_type[len('class '):]

        if current_input_type in mapping_dict:
            if mapping_dict[current_input_type]['specific_usage'] not in {None, 'None'}:
                args.append(mapping_dict[current_input_type]['specific_usage'])
            else:
                if mapping_dict[current_input_type]['parameter'] not in {None, 'None'}:
                    args.append(mapping_dict[current_input_type]['parameter'])
                    
        # Check if the input is an output and is required
        if input.isOutput() and input.isRequired():
            # Change the current argument to 'result_image'
            args[-1] = 'result_image'

    # Join the arguments with ', ' and add them to the function call
    code += ', '.join(args) + '))\n'
    return code

print(op_calling_and_running(op_info,mapping_dict))
def type_conversion():
    return 'result_image = ij.py.from_java(result_image)\n'
print(type_conversion())

def normalization_for_PIL_image():
    return 'result_image = ((result_image - result_image.min()) / (result_image.max() - result_image.min())* 255).astype(\'uint8\')\n'
print(normalization_for_PIL_image())

def display_image_information():
    return textwrap.dedent('''\
        def image_info(image):
            print("--- After Edge Preserve Smoothing ---")
            ij.py.show(image)
            print(f" type: {type(image)}")
            print(f"dtype: {image.dtype if hasattr(image, 'dtype') else 'N/A'}")
            print(f"shape: {image.shape}")
            print ("--------------------------------")
        image_info(result_image);
    ''')
print(display_image_information())

def add_metadata_for_output():
    return textwrap.dedent('''\
        result_image = {
          'value': result_image,
          'dataType': 'numpy.ndarray',
          'metadata': {
            'colorChannel': 'grayscale',
            'channelOrder': 'channelLast',
            'isMiniBatched': False,
            'intensityRange': '0-255',
            'device': 'cpu'
          }
        }
    ''')
print(add_metadata_for_output())


import imagej
import scyjava as sj
ij = imagej.init('sc.fiji:fiji:2.14.0')

#Get input image
vp_input_image = n_1_image
process_image = vp_input_image.get('value')

Op_name = 'filter.median'

jimage = ij.py.to_java(process_image)
result_image = ij.op().run('create.img',jimage)

result_image= ij.op().run(Op_name, ij.py.jargs(result_image, process_image, HyperSphereShape(range), None))

result_image = ij.py.from_java(result_image)

result_image = ((result_image - result_image.min()) / (result_image.max() - result_image.min())* 255).astype('uint8')

def image_info(image):
    print("--- After Edge Preserve Smoothing ---")
    ij.py.show(image)
    print(f" type: {type(image)}")
    print(f"dtype: {image.dtype if hasattr(image, 'dtype') else 'N/A'}")
    print(f"shape: {image.shape}")
    print ("--------------------------------")
image_info(result_image);

result_image = {
  'value': result_image,
  'dataType': 'numpy.ndarray',
  'metadata': {
    'colorChannel': 'grayscale',
    'channel

### Main Calling Method

In [107]:
def generate_code_snippet(op_name, mapping_dict):
    """
    Generate a code snippet for the given operation name and mapping dictionary.
    
    Parameters:
    op_name (str): The name of the operation.
    mapping_dict (dict): The mapping dictionary.

    Returns:
    str: The generated code snippet.
    """
    op_info = get_op_info(op_name)
    required_parameters = get_op_required_parameters(op_info)
    
    code_snippet = import_part()
    code_snippet += get_input_image()
    code_snippet += get_op_name(op_name)
    code_snippet += import_java_class_by_jimport(required_parameters, mapping_dict)
    
    if is_empty_result_image_required(op_info):
        code_snippet += prepare_empty_result_image()
    
    code_snippet += op_calling_and_running(op_info, mapping_dict)
    code_snippet += type_conversion()
    code_snippet += normalization_for_PIL_image()
    code_snippet += display_image_information()
    code_snippet += add_metadata_for_output()
    
    return code_snippet
print(generate_code_snippet(op_name,mapping_dict))


import imagej
import scyjava as sj
ij = imagej.init('sc.fiji:fiji:2.14.0')
#Get input image
vp_input_image = n_1_image
process_image = vp_input_image.get('value')
Op_name = 'filter.median'
HyperSphereShape = sj.jimport("net.imglib2.algorithm.neighborhood.HyperSphereShape")
jimage = ij.py.to_java(process_image)
result_image = ij.op().run('create.img',jimage)
result_image= ij.op().run(Op_name, ij.py.jargs(result_image, process_image, HyperSphereShape(range), None))
result_image = ij.py.from_java(result_image)
result_image = ((result_image - result_image.min()) / (result_image.max() - result_image.min())* 255).astype('uint8')
def image_info(image):
    print("--- After Edge Preserve Smoothing ---")
    ij.py.show(image)
    print(f" type: {type(image)}")
    print(f"dtype: {image.dtype if hasattr(image, 'dtype') else 'N/A'}")
    print(f"shape: {image.shape}")
    print ("--------------------------------")
image_info(result_image);
result_image = {
  'value': result_image,
  'dataType': '

### Full Op list

In [31]:
ops_list = [op for op in ij.op().ops()]
ops_list

['coloc.icq',
 'coloc.kendallTau',
 'coloc.maxTKendallTau',
 'coloc.pValue',
 'coloc.pearsons',
 'convert.bit',
 'convert.cfloat32',
 'convert.cfloat64',
 'convert.clip',
 'convert.copy',
 'convert.float32',
 'convert.float64',
 'convert.imageType',
 'convert.int16',
 'convert.int32',
 'convert.int64',
 'convert.int8',
 'convert.normalizeScale',
 'convert.scale',
 'convert.uint12',
 'convert.uint128',
 'convert.uint16',
 'convert.uint2',
 'convert.uint32',
 'convert.uint4',
 'convert.uint64',
 'convert.uint8',
 'copy.img',
 'copy.imgLabeling',
 'copy.iterableInterval',
 'copy.labelingMapping',
 'copy.rai',
 'copy.type',
 'create.img',
 'create.imgFactory',
 'create.imgLabeling',
 'create.imgPlus',
 'create.integerType',
 'create.kernel',
 'create.kernel2ndDerivBiGauss',
 'create.kernelBiGauss',
 'create.kernelDiffraction',
 'create.kernelGabor',
 'create.kernelGaborComplexDouble',
 'create.kernelGaborDouble',
 'create.kernelGauss',
 'create.kernelLog',
 'create.kernelSobel',
 'create.l

### Save ops by category

In [38]:
print(type(ops_list[0]))

<java class 'java.lang.String'>


In [40]:
ops_dict = {}
for op in ops_list:
    op_str = str(op)  # convert Java string to Python string
    split_op = op_str.split('.')
    if len(split_op) > 1:
        category, specific_op = split_op[0], '.'.join(split_op[1:])
        if category not in ops_dict:
            ops_dict[category] = []
        ops_dict[category].append(specific_op)

import pprint

# Assuming ops_dict is your dictionary
pprint.pprint(ops_dict)



{'coloc': ['icq', 'kendallTau', 'maxTKendallTau', 'pValue', 'pearsons'],
 'convert': ['bit',
             'cfloat32',
             'cfloat64',
             'clip',
             'copy',
             'float32',
             'float64',
             'imageType',
             'int16',
             'int32',
             'int64',
             'int8',
             'normalizeScale',
             'scale',
             'uint12',
             'uint128',
             'uint16',
             'uint2',
             'uint32',
             'uint4',
             'uint64',
             'uint8'],
 'copy': ['img',
          'imgLabeling',
          'iterableInterval',
          'labelingMapping',
          'rai',
          'type'],
 'create': ['img',
            'imgFactory',
            'imgLabeling',
            'imgPlus',
            'integerType',
            'kernel',
            'kernel2ndDerivBiGauss',
            'kernelBiGauss',
            'kernelDiffraction',
            'kernelGabor',
           

### OpInfo's cInfo: Title, Name, Namespace, Type

In [108]:
title = op_info[0].cInfo().getTitle()
print(title)

name = op_info[0].getName()
print(name)

simple_name = op_info[0].getSimpleName()
print(simple_name)

namespace = op_info[0].getNamespace()
print(namespace)

inputs = op_info[0].inputs()
print(inputs) # Default with List type
print(inputs[1].getType())

outputs = op_info[0].outputs()
print(outputs) # Default with List type
print(outputs[0].getType())

command:net.imagej.ops.filter.median.DefaultMedianFilter
filter.median
median
filter
[out: required=true, persisted=true, in: required=true, persisted=true, shape: required=true, persisted=true, outOfBoundsFactory: required=false, persisted=true, default=net.imglib2.outofbounds.OutOfBoundsBorderFactory@9accff0]
interface net.imglib2.RandomAccessibleInterval
[out: required=true, persisted=true]
interface net.imglib2.IterableInterval


### Mapping dict full

In [17]:
import pandas as pd
def mapping_xlsx_to_mapping_dict(file_path):
    df = pd.read_excel(file_path,index_col = 0)
    data_dict = df.to_dict('index')
    return data_dict

mapping_dict = mapping_xlsx_to_mapping_dict(type_mapping_table_file)
import pprint

# Assuming ops_dict is your dictionary
pprint.pprint(mapping_dict)

{'boolean': {'datatype': 'bool',
             'default_value': 'TRUE',
             'jimport': 'None',
             'parameter': 'None',
             'specific_usage': 'None'},
 'double': {'datatype': 'float',
            'default_value': 2.1,
            'jimport': 'None',
            'parameter': 'None',
            'specific_usage': 'None'},
 'float': {'datatype': 'float',
           'default_value': 2.1,
           'jimport': 'None',
           'parameter': 'None',
           'specific_usage': 'None'},
 'int': {'datatype': 'integer',
         'default_value': 5,
         'jimport': 'None',
         'parameter': 'None',
         'specific_usage': 'None'},
 'java.lang.Boolean': {'datatype': 'bool',
                       'default_value': 'TRUE',
                       'jimport': 'None',
                       'parameter': 'None',
                       'specific_usage': 'None'},
 'java.lang.Double': {'datatype': 'float',
                      'default_value': 2.1,
                   